In [1]:
import numpy as np
import os

#os.environ['CUDA_VISIBLE_DEVICES'] = '1'
#os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
actions = [
    'come',
    'away',
    'spin'
]

data = np.concatenate([
    np.load('dataset/seq_come_1627646273.npy'),
    np.load('dataset/seq_away_1627646273.npy'),
    np.load('dataset/seq_spin_1627646273.npy')
], axis=0)

data.shape

(1272, 30, 100)

In [3]:
x_data = data[:, :, :-1]
labels = data[:, 0, -1]

print(x_data.shape)
print(labels.shape)

(1272, 30, 99)
(1272,)


In [4]:
from tensorflow.keras.utils import to_categorical

y_data = to_categorical(labels, num_classes=len(actions))
y_data.shape

(1272, 3)

In [5]:
from sklearn.model_selection import train_test_split

x_data = x_data.astype(np.float32)
y_data = y_data.astype(np.float32)

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=2021)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(1144, 30, 99) (1144, 3)
(128, 30, 99) (128, 3)


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(64, activation='relu', input_shape=x_train.shape[1:3]),
    Dense(32, activation='relu'),
    Dense(len(actions), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                41984     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                                 
Total params: 44,163
Trainable params: 44,163
Non-trainable params: 0
_________________________________________________________________


In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

history = model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=200,
    callbacks=[
        ModelCheckpoint('models/model.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=50, verbose=1, mode='auto'),
        EarlyStopping(patience=10, monitor='val_acc')
    ]
)

Epoch 1/200
34/36 [===========================>..] - ETA: 0s - loss: 62.4213 - acc: 0.4449
Epoch 1: val_acc improved from -inf to 0.57812, saving model to model\model.h5
36/36 [==============================] - 2s 15ms/step - loss: 60.2576 - acc: 0.4519 - val_loss: 29.8465 - val_acc: 0.5781 - lr: 0.0010
Epoch 2/200
33/36 [==========================>...] - ETA: 0s - loss: 36.7061 - acc: 0.6127
Epoch 2: val_acc improved from 0.57812 to 0.67969, saving model to model\model.h5
36/36 [==============================] - 0s 8ms/step - loss: 35.2189 - acc: 0.6233 - val_loss: 29.0894 - val_acc: 0.6797 - lr: 0.0010
Epoch 3/200
31/36 [========================>.....] - ETA: 0s - loss: 97.7527 - acc: 0.5020
Epoch 3: val_acc did not improve from 0.67969
36/36 [==============================] - 0s 8ms/step - loss: 96.0692 - acc: 0.5035 - val_loss: 55.1054 - val_acc: 0.4453 - lr: 0.0010
Epoch 4/200
33/36 [==========================>...] - ETA: 0s - loss: 28.2651 - acc: 0.6061
Epoch 4: val_acc improved 

In [8]:
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots(figsize=(16, 10))
acc_ax = loss_ax.twinx()

loss_ax.plot(history.history['loss'], 'y', label='train loss')
loss_ax.plot(history.history['val_loss'], 'r', label='val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(history.history['acc'], 'b', label='train acc')
acc_ax.plot(history.history['val_acc'], 'g', label='val acc')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='upper left')

plt.show()

In [9]:
from sklearn.metrics import multilabel_confusion_matrix
from tensorflow.keras.models import load_model

model = load_model('models/model.h5')

y_pred = model.predict(x_val)

multilabel_confusion_matrix(np.argmax(y_val, axis=1), np.argmax(y_pred, axis=1))

4/4 [==============================] - 0s 3ms/step


array([[[94,  0],
        [ 1, 33]],

       [[81,  0],
        [ 0, 47]],

       [[80,  1],
        [ 0, 47]]], dtype=int64)